# Writing CNN layers

## 2D CNN

#### Convolution

In [ ]:
import numpy as np


def conv2d(image, kernel, stride, padding):
    """
    :param image: 
    :param kernel: convolutional kernel
    :param stride: steps to move kernel
    :param padding: number of paddings on each side
    :return: 
    """
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape
    
    # Output dimensions
    out_height = int((image_height - kernel_height + 2 * padding) / stride + 1)
    out_width = int((image_width - kernel_width + 2 * padding) / stride + 1)
    
    # Apply padding
    padded_image = np.pad(image, ((padding, padding), (padding, padding)), mode='constant', constant_values=0)
    
    # Initialize the output feature map
    output = np.zeros((out_height, out_width))
    
    # Perform convolution
    for y in range(0, out_height):
        for x in range(0, out_width):
            output[y, x] = np.sum(padded_image[y*stride:y*stride+kernel_height, x*stride:x*stride+kernel_width] * kernel)
    return output

#### Pooling

In [ ]:
def max_pooling2d(feature_map, size, stride):
    """
    Apply max pooling operation to a feature map.
    """
    pool_height, pool_width = size
    out_height = int((feature_map.shape[0] - pool_height) / stride + 1)
    out_width = int((feature_map.shape[1] - pool_width) / stride + 1)
    
    output = np.zeros((out_height, out_width))
    
    for y in range(out_height):
        for x in range(out_width):
            output[y, x] = np.max(feature_map[y*stride:y*stride+pool_height, x*stride:x*stride+pool_width])
    return output

#### Example Model

In [ ]:

def relu(x):
    """
    ReLU activation function.
    """
    return np.maximum(0, x)

def softmax(x):
    """
    Softmax activation function for the output layer.
    """
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

class SimpleCNN:
    def __init__(self):
        self.conv_filter = np.random.rand(3, 3) - 0.5
        self.fc_weights = np.random.rand(10, 36) - 0.5  # Assuming flattened 6x6 pooling output for 10 classes
        self.fc_bias = np.random.rand(10) - 0.5
        
    def forward(self, image):
        """
        Forward pass through the CNN.
        """
        conv_out = relu(conv2d(image, self.conv_filter, stride=1, padding=1))
        pooled_out = max_pooling2d(conv_out, size=(2, 2), stride=2)
        flattened = pooled_out.flatten()
        fc_out = softmax(np.dot(self.fc_weights, flattened) + self.fc_bias)
        return fc_out

# Example usage
if __name__ == "__main__":
    image = np.random.rand(28, 28)  # Example image
    cnn = SimpleCNN()
    output = cnn.forward(image)
    print("Output:", output)

## 3D 

#### Convolution

In [ ]:
def convolve3d(input_volume, filter, stride, padding):
    """
    Apply a 3D convolution operation to an input volume using the given filter.
    """
    input_depth, input_height, input_width = input_volume.shape
    filter_depth, filter_height, filter_width = filter.shape
    
    # Output dimensions
    out_depth = int((input_depth - filter_depth + 2 * padding) / stride + 1)
    out_height = int((input_height - filter_height + 2 * padding) / stride + 1)
    out_width = int((input_width - filter_width + 2 * padding) / stride + 1)
    
    # Apply padding
    padded_input = np.pad(input_volume, [(padding, padding), (padding, padding), (padding, padding)], mode='constant', constant_values=0)
    
    # Initialize the output feature map
    output = np.zeros((out_depth, out_height, out_width))
    
    # Perform 3D convolution
    for z in range(out_depth):
        for y in range(out_height):
            for x in range(out_width):
                output[z, y, x] = np.sum(padded_input[z*stride:z*stride+filter_depth,
                                                      y*stride:y*stride+filter_height,
                                                      x*stride:x*stride+filter_width] * filter)
    return output


#### Pooling

In [ ]:
def max_pooling3d(input_volume, size, stride):
    """
    Apply max pooling operation to a 3D input volume.
    """
    pool_depth, pool_height, pool_width = size
    out_depth = int((input_volume.shape[0] - pool_depth) / stride + 1)
    out_height = int((input_volume.shape[1] - pool_height) / stride + 1)
    out_width = int((input_volume.shape[2] - pool_width) / stride + 1)
    
    output = np.zeros((out_depth, out_height, out_width))
    
    for z in range(out_depth):
        for y in range(out_height):
            for x in range(out_width):
                output[z, y, x] = np.max(input_volume[z*stride:z*stride+pool_depth,
                                                      y*stride:y*stride+pool_height,
                                                      x*stride:x*stride+pool_width])
    return output

#### Example Model

In [ ]:
import numpy as np

def relu(x):
    """ReLU activation function."""
    return np.maximum(0, x)

def softmax(x):
    """Softmax activation function."""
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x, axis=0)

class Simple3DCNN:
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape  # Expected input shape: (depth, height, width)
        self.num_classes = num_classes
        
        # Initialize weights for a single 3D convolutional filter
        self.conv_filter = np.random.randn(3, 3, 3) - 0.5
        
        # Initialize weights and biases for the fully connected layer
        # Assume flattened output from pooling layer for simplicity
        self.fc_weights = np.random.randn(num_classes, 108) - 0.5  # Example shape
        self.fc_bias = np.random.randn(num_classes) - 0.5
    
    def forward(self, input_volume):
        """Forward pass through the 3D CNN."""
        # Convolutional layer
        conv_out = relu(convolve3d(input_volume, self.conv_filter, stride=1, padding=1))
        
        # Pooling layer
        pooled_out = max_pooling3d(conv_out, size=2, stride=2)
        
        # Flatten
        flattened = pooled_out.flatten()
        
        # Fully connected layer
        fc_out = softmax(np.dot(self.fc_weights, flattened) + self.fc_bias)
        
        return fc_out

In [ ]:
# Example usage
if __name__ == "__main__":
    # Dummy volumetric data
    input_volume = np.random.rand(10, 32, 32)  # Example 3D input (depth, height, width)
    model = Simple3DCNN(input_shape=input_volume.shape, num_classes=3)
    output = model.forward(input_volume)
    print("Output:", output)